### load packages

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats

import os
import glob
from functools import reduce

import matplotlib.pyplot as plt
import pylab 

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from qq_plot import *


In [38]:
# use R inside python 
import rpy2
import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector
from rpy2.robjects.packages import importr

from rpy2.robjects import numpy2ri
numpy2ri.activate()

from rpy2.robjects import pandas2ri
pandas2ri.activate()

utils = rpackages.importr('utils')
utils.chooseCRANmirror(ind=1)

# install packages
packnames = ('betategarch', 'rugarch')
utils.install_packages(StrVector(packnames))

R[write to console]: versuche URL 'https://cloud.r-project.org/bin/macosx/contrib/4.1/betategarch_3.3.tgz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 127596 bytes (124 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[w


The downloaded binary packages are in
	/var/folders/wz/thdn6rxd3zxcr124pwqxftrc0000gn/T//Rtmpi9vxyR/downloaded_packages


<rpy2.rinterface_lib.sexp.NULLType object at 0x7fa670fb6680> [RTYPES.NILSXP]

### load data

In [4]:
# path to the directory 
all_files = glob.glob(os.path.join('Prices', '*.csv'))
# file (parameter) names
name_list = []
for name in sorted(os.listdir('./Prices/')):
    if name.endswith('.csv'):
        name = name.replace('.csv', '')
        name_list.append(name)

# load data for close price only 
dfs = []
for file, i in zip(sorted(all_files), range(len(name_list))):
    df = pd.read_csv(file)[['time','high']]
    if df['high'].iloc[-5] != 0 and df['high'].iloc[-2] != 0:
        df = df.rename(columns={'high':name_list[i], 
                                'time':'date'})
        df['date'] = pd.to_datetime(df['date'],unit='s')
        df.set_index(df['date'], inplace=True)
        df = df.drop(columns=['date'])
        dfs.append(df)

In [5]:
price_df = pd.concat(dfs, axis=1)
price_df = price_df.fillna(0)

### check each time series's period

In [6]:
max_dates = dict()
for crypto in price_df.columns:
    num_zeros = price_df[crypto].loc[price_df[crypto] == 0].count()
    if num_zeros != 0:
        max_date = price_df[crypto].loc[price_df[crypto] == 0].index.max().strftime('%Y-%m-%d')
        min_date = price_df[crypto].loc[price_df[crypto] == 0].index.min().strftime('%Y-%m-%d')
        max_dates.update({crypto: max_date})
    else:
        max_dates.update({crypto: 'NaT'})
    # print('number of 0s: %s for %s from %s to %s' % (num_zeros, crypto, min_date, max_date))

In [7]:
def get_crypto_after_zero(price_df, crypto, max_dates):
    if max_dates[crypto] == 'NaT':
        df =  price_df[crypto]
    else:
        df = price_df[crypto].loc[max_dates[crypto]:][1:]
    return df

### beta-skew-t egarch

In [8]:
# load R packages
betategarch = importr('betategarch')

def one_comp_betategarch(data):
    """
    :param data: should be an array 
    :return: estimation result
    :return: fitted volatility (conditional standard deviation)
    """
    array_data = np.array(data)
    one_comp = betategarch.tegarch(**{'y': array_data, 'skew':True}) 
    one_stdev = betategarch.fitted_tegarch(one_comp)
    print(data.name, one_comp[13])
    return one_comp, one_stdev

def two_comp_betategarch(data):
    """
    :param data: should be an array 
    :return: estimation result
    :return: fitted volatility (conditional standard deviation)
    """
    array_data = np.array(data)
    two_comp = betategarch.tegarch(**{'y': array_data, 'components':2, 'skew':True, 'asym':True}) 
    two_stdev = betategarch.fitted_tegarch(two_comp)
    print(data.name, two_comp[13])
    return two_comp, two_stdev

def zoo_to_df(zoo, index):
    zoo_df = pd.DataFrame(zoo, 
                           index=index,
                           columns=['return_y', 
                                    'fitted_scale_sigma', 
                                    'fitted_conditional_stdev',
                                    'fitted_log-scale_lambda', 
                                    'lambdadagger', 
                                    'lambda2dagger',
                                    'score_u', 
                                    'residuals_epsilon', 
                                    'residstd'])

    zoo_df['snr'] = np.exp(zoo_df['lambda2dagger']) / (np.exp(zoo_df['lambdadagger']) + np.exp(zoo_df['lambda2dagger']))
    zoo_df['snr'] = (zoo_df['snr'] / zoo_df['snr'][0])*100
    
    zoo_df['return_sign'] = np.where(zoo_df['return_y']<0, 0, 1)

    return zoo_df

def one_zoo_df(zoo, index):
    zoo_df = pd.DataFrame(zoo, 
                           index=index,
                           columns=['return_y', 
                                    'fitted_scale_sigma', 
                                    'fitted_conditional_stdev',
                                    'fitted_log-scale_lambda', 
                                    'lambdadagger', 
                                    'score_u', 
                                    'residuals_epsilon', 
                                    'residstd'])


    return zoo_df

def plot_return_2comp(zoo_df, save_path):
    fig = make_subplots(rows=2, cols=1, 
                        shared_xaxes=True, 
                        vertical_spacing=0.05)

    fig['layout'].update(height=600, width=1200, 
                        title='',
                        showlegend=False,
                        font=dict(family='Times New Roman', size=16))

    # fig.append_trace(go.Scatter(
    #     y=zoo_df['return_y']*100, 
    #     x=zoo_df.index, 
    #     line=dict(color='black', width=1)       
    #     ), row=1, col=1)
    fig.append_trace(go.Scatter(
        y=(zoo_df['lambdadagger']*100), 
        x=zoo_df.index,
        line=dict(color='red', width=1)   
        ), row=1, col=1)
    fig.append_trace(go.Scatter(
        y=(zoo_df['lambda2dagger']*100),
        x=zoo_df.index,
        line=dict(color='blue', width=1)
        ), row=2, col=1)

    fig['layout']['xaxis2'].update(title='Date', tickformat="%b\n%Y")

    # fig['layout']['yaxis1'].update(title='Log return %')
    fig['layout']['yaxis1'].update(title='Long-run <br> volatility component')
    fig['layout']['yaxis2'].update(title='Short-run <br> volatility component')

    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False)
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False)


    fig.update_layout({'plot_bgcolor': 'rgba(0,0,0,0)',
                    'paper_bgcolor': 'rgba(0,0,0,0)'},
                    font_color='black')

    fig.show()
    fig.write_image(save_path)


### acf

In [9]:
from statsmodels.tsa.stattools import acf, pacf

def create_corr_plot(series, date, save_path, plot_pacf=False):
    corr_array = pacf(series.dropna(), alpha=0.05, nlags=50) if plot_pacf else acf(series.dropna(), alpha=0.05, nlags=50)
    lower_y = corr_array[1][:,0] - corr_array[0]
    upper_y = corr_array[1][:,1] - corr_array[0]
   
    fig = make_subplots(rows=2, cols=1, vertical_spacing=0.18)
    
    fig.add_trace(go.Scatter(y=series,
                             x=date,
                             mode='lines',
                             line=dict(color='blue', width=1)
                             ), 
                  row=1, col=1)
    
    fig['layout'].update(height=800, width=1200,
                    title='',
                    showlegend=False,
                    font=dict(family='Times New Roman', size=14))
    [fig.add_trace(go.Scatter(y=(0,corr_array[0][x]),
                             x=(x,x),
                             mode='lines',
                             line=dict(color='black', width=2)
                             ), 
                   row=2, col=1)
     for x in range(len(corr_array[0]))]
    
    fig.add_trace(go.Scatter(y=corr_array[0],
                             x=np.arange(len(corr_array[0])),
                             mode='markers',
                             marker=dict(color='blue', size=8)
                             ), 
                  row=2, col=1)
    
    fig.add_trace(go.Scatter(y=upper_y,
                             x=np.arange(len(corr_array[0])),
                             mode='lines',
                             line=dict(color='rgba(173, 216, 230,0.4)', width=1)
                             ), 
                  row=2, col=1)
    fig.add_trace(go.Scatter(x=np.arange(len(corr_array[0])), y=lower_y,
                             mode='lines',
                             fillcolor='rgba(173, 216, 230,0.4)',
                             fill='tonexty', 
                             line_color='rgba(173, 216, 230,0.4)'), 
                  row=2, col=1)

    fig.update_xaxes(title='Date', dtick = 'M3', row=1, col=1,)
    fig.update_xaxes(title='Lags', range=[-1,52], row=2, col=1,)
    fig.update_yaxes(title='Log returns', zerolinecolor='black', row=1, col=1,)
    fig.update_yaxes(title='ACF', zerolinecolor='black', row=2, col=1,)
    
    fig.update_xaxes(showline=True, linewidth=1, 
                 linecolor='black', 
                 mirror=True,
                 tickformat="%b\n%Y", 
                 showgrid=False)
    fig.update_yaxes(showline=True, linewidth=1, 
                 linecolor='black', 
                 mirror=True, 
                 showgrid=False)
    fig.update_layout({'plot_bgcolor': 'rgba(0,0,0,0)',
                    'paper_bgcolor': 'rgba(0,0,0,0)'},                  
                    font_color='black')

    fig.show()
    fig.write_image(save_path)


In [11]:
eth_price = get_crypto_after_zero(price_df, 'ETH', max_dates)
eth_return = (np.log(eth_price) - np.log(eth_price.shift(1)))['2020-06-01':'2023-01-16']


In [12]:
eth1comp = one_comp_betategarch(eth_return)
eth2comp = two_comp_betategarch(eth_return)

ETH [1] "relative convergence (4)"

ETH [1] "relative convergence (4)"



In [13]:
eth_zoo1 = one_zoo_df(zoo=betategarch.fitted_tegarch(eth1comp[0], 
                                                   verbose=True), 
                    index=eth_return.index)['2021-01-01':'2023-01-10']


In [14]:
eth_zoo = zoo_to_df(zoo=betategarch.fitted_tegarch(eth2comp[0], 
                                                   verbose=True), 
                    index=eth_return.index)['2021-01-01':'2023-01-10']


In [15]:
eth_zoo.to_csv('eth_zoo.csv')

In [16]:
plot_return_2comp(eth_zoo, save_path='./image/eth_long_short.pdf')

In [17]:
def auto_correlation(series, plot_pacf=True):
    corr_array = pacf(series.dropna(), alpha=0.05, nlags=50) if plot_pacf else acf(series.dropna(), alpha=0.05, nlags=50, fft=False)
    lower_y = corr_array[1][:,0] - corr_array[0]
    upper_y = corr_array[1][:,1] - corr_array[0]
    return corr_array, lower_y, upper_y

acf_eth_retrun = auto_correlation(eth_return)
acf_abs_eth_retrun = auto_correlation(abs(eth_return))
acf_squared_eth_return = auto_correlation(np.square(eth_return))

fig = make_subplots(rows=2, cols=3,
                    shared_xaxes=False,
                    specs=[[{'colspan': 3, 'rowspan':1}, None, None],  [{},{},{}]],
                    subplot_titles=(['',  'Log returns', 'Absolute log returns', 'Squared log returns']),
                    vertical_spacing=0.15,
                    horizontal_spacing=0)

fig['layout'].update(height=800, width=1200,
                    title='',
                    showlegend=False,
                    font=dict(family='Times New Roman', size=12))

# eth 
fig.add_traces(go.Scatter(y=eth_return, x=eth_return.index,line=dict(color='#000000', width=1)), rows=1, cols=1)

fig.add_traces([go.Scatter(x=(x,x), y=(0,acf_eth_retrun[0][0][x]), mode='lines',line_color='#3f3f3f') 
                for x in range(len(acf_eth_retrun[0][0]))], 
               rows=2, cols=1)
fig.add_traces([go.Scatter(x=np.arange(len(acf_eth_retrun[0][0])), y=acf_eth_retrun[0][0], 
                           mode='markers',marker_color='#000000', marker_size=5),
                go.Scatter(x=np.arange(len(acf_eth_retrun[0][0])), y=acf_eth_retrun[2], mode='lines', 
                           line_color='rgba(255,255,255,0)'),
                go.Scatter(x=np.arange(len(acf_eth_retrun[0][0])), y=acf_eth_retrun[1], mode='lines',
                           fillcolor='rgba(0, 0, 0,0.15)',fill='tonexty', line_color='rgba(255,255,255,0)')],
               rows=2, cols=1)

fig.add_traces([go.Scatter(x=(x,x), y=(0,acf_abs_eth_retrun[0][0][x]), mode='lines',
                           line_color='#3f3f3f') for x in range(len(acf_abs_eth_retrun[0][0]))], 
               rows=2, cols=2)
fig.add_traces([go.Scatter(x=np.arange(len(acf_abs_eth_retrun[0][0])), y=acf_abs_eth_retrun[0][0],
                           mode='markers',marker_color='#000000', marker_size=5),
                go.Scatter(x=np.arange(len(acf_abs_eth_retrun[0][0])), y=acf_abs_eth_retrun[2], 
                           mode='lines', line_color='rgba(255,255,255,0)'),
                go.Scatter(x=np.arange(len(acf_abs_eth_retrun[0][0])), y=acf_abs_eth_retrun[1], 
                           mode='lines', fillcolor='rgba(0, 0, 0,0.15)',fill='tonexty', line_color='rgba(255,255,255,0)')],
               rows=2, cols=2)

fig.add_traces([go.Scatter(x=(x,x), y=(0,acf_squared_eth_return[0][0][x]), mode='lines',
                           line_color='#3f3f3f') for x in range(len(acf_squared_eth_return[0][0]))], 
               rows=2, cols=3)
fig.add_traces([go.Scatter(x=np.arange(len(acf_squared_eth_return[0][0])), 
                           y=acf_squared_eth_return[0][0], 
                           mode='markers', marker_color='#000000', marker_size=5),
                go.Scatter(x=np.arange(len(acf_squared_eth_return[0][0])), y=acf_squared_eth_return[2], 
                           mode='lines', line_color='rgba(255,255,255,0)'),
                go.Scatter(x=np.arange(len(acf_squared_eth_return[0][0])), y=acf_squared_eth_return[1], 
                           mode='lines',fillcolor='rgba(0, 0, 0,0.15)',fill='tonexty',
                           line_color='rgba(255,255,255,0)')],
               rows=2, cols=3)

fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True,showgrid=False)
fig.update_yaxes(zerolinecolor='#000000',
                 showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False)

fig['layout']['yaxis1'].update(title_text='Log returns')
fig['layout']['yaxis2'].update(title_text='ACF')
fig['layout']['yaxis3'].update(showticklabels = False)
fig['layout']['yaxis4'].update(showticklabels = False)

fig['layout']['xaxis1'].update(title_text='Date', tickformat="%B \n%Y")
fig['layout']['xaxis2'].update(title_text='Lags', range=[-1,52])
fig['layout']['xaxis3'].update(title_text='Lags', range=[-1,52])
fig['layout']['xaxis4'].update(title_text='Lags', range=[-1,52])

fig.update_layout({'plot_bgcolor': 'rgba(0,0,0,0)',
                   'paper_bgcolor': 'rgba(0,0,0,0)'},
                  font_color='#000000',
                  margin=dict(l=0,r=0,b=0,t=0))

fig.show()
fig.write_image('./image/acf_eth.pdf')


In [18]:
acf_eth_long = auto_correlation(eth_zoo['lambdadagger'])
acf_abs_eth_short = auto_correlation(abs(eth_zoo['lambda2dagger']))

fig = make_subplots(rows=1, cols=2,
                    horizontal_spacing=0.01)

fig['layout'].update(height=400, width=1200, showlegend=False,
                    font=dict(family='Times New Roman', size=12))

fig.add_traces([go.Scatter(x=(x,x), y=(0,acf_eth_long[0][0][x]), mode='lines', line_color='#3f3f3f') 
                for x in range(len(acf_eth_long[0][0]))], 
               rows=1, cols=1)
fig.add_traces([go.Scatter(x=np.arange(len(acf_eth_long[0][0])), y=acf_eth_long[0][0], 
                           mode='markers',marker_color='#ff0101', marker_size=5),
                go.Scatter(x=np.arange(len(acf_eth_long[0][0])), y=acf_eth_long[2], mode='lines', 
                           line_color='rgba(255,255,255,0)'),
                go.Scatter(x=np.arange(len(acf_eth_long[0][0])), y=acf_eth_long[1], mode='lines',
                           fillcolor='rgba(0, 0, 0,0.15)',fill='tonexty', line_color='rgba(255,255,255,0)')],
               rows=1, cols=1)

fig.add_traces([go.Scatter(x=(x,x), y=(0,acf_abs_eth_short[0][0][x]), mode='lines',
                           line_color='#3f3f3f') for x in range(len(acf_abs_eth_short[0][0]))], 
               rows=1, cols=2)
fig.add_traces([go.Scatter(x=np.arange(len(acf_abs_eth_short[0][0])), y=acf_abs_eth_short[0][0],
                           mode='markers',marker_color='#0000ff', marker_size=5),
                go.Scatter(x=np.arange(len(acf_abs_eth_short[0][0])), y=acf_abs_eth_short[2], 
                           mode='lines', line_color='rgba(255,255,255,0)'),
                go.Scatter(x=np.arange(len(acf_abs_eth_short[0][0])), y=acf_abs_eth_short[1], 
                           mode='lines', fillcolor='rgba(0, 0, 0,0.15)',fill='tonexty', line_color='rgba(255,255,255,0)')],
               rows=1, cols=2)


fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True,showgrid=False)
fig.update_yaxes(zerolinecolor='#000000',
                 showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False)

fig['layout']['yaxis1'].update(title_text='ACF', range=[-0.2,1.1])
fig['layout']['yaxis2'].update(title_text='', range=[-0.2,1.1], showticklabels = False)

fig['layout']['xaxis1'].update(title_text='Lags', range=[-1,52])
fig['layout']['xaxis2'].update(title_text='Lags', range=[-1,52])

fig.update_layout({'plot_bgcolor': 'rgba(0,0,0,0)',
                   'paper_bgcolor': 'rgba(0,0,0,0)'},
                  font_color='#000000',
                  margin=dict(l=0,r=0,b=0,t=0))

fig.show()
fig.write_image('./image/acf_eth_long_short.pdf')


### DL

In [19]:
return_df_col = pd.DataFrame(eth_zoo['return_y'])

return_df_col['color'] = np.where(return_df_col<0, ' rgba(255,1,1,0.4)', 'rgba(1,129,1,0.4)')

fig = make_subplots(specs=[[{"secondary_y": True}],[{"secondary_y": True}]], 
                    rows=2, cols=1, 
                    shared_xaxes=True, 
                    vertical_spacing=0.01)

fig['layout'].update(height=600, width=1200, showlegend=False,
                    font=dict(family='Times New Roman', size=16))
fig.add_trace(
    go.Scatter(
           x=return_df_col.index,
           y=eth_zoo['snr'],
          mode='lines',
          line=dict(color='rgba(0,0,0,1)', width=1.5)), 
    secondary_y=False, row=1, col=1)

fig.add_trace(
    go.Bar(x=return_df_col.index,
           y=return_df_col['return_y']*100,
           marker_color=return_df_col['color'], 
           marker_line_width=0), 
    secondary_y=True, row=1, col=1)

fig.add_vline(x='2022-09-15', 
                line_dash='dot',
                line_color='black',
                line_width=0.5)

fig.add_trace(
    go.Scatter(
           x=return_df_col.index,
           y=eth_zoo['snr'],
          mode='lines',
          line=dict(color='rgba(0,0,0,1)', width=1.5)), 
    secondary_y=False, row=2, col=1)

fig.add_trace(
    go.Scatter(
           x=return_df_col.index,
           y=(eth_zoo['return_y']*100).rolling(7).mean(),
          mode='lines',
          line=dict(color='rgba(0,0,255,1)', width=1.5, dash='dot')), 
    secondary_y=True, row=2, col=1)

fig.add_vline(x='2022-09-15', 
                line_dash='dot',
                line_color='black',
                line_width=0.5)

fig.update_layout(bargap=0, bargroupgap=0)

fig.update_layout({'plot_bgcolor': 'rgba(0,0,0,0)',
                    'paper_bgcolor': 'rgba(0,0,0,0)'},                  
                    font_color='black')

fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True,showgrid=False,  tickformat='%b\n%Y')
fig.update_yaxes(zerolinecolor='#000000',
                 showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False)

# fig['layout']['yaxis1'].update(title='')
# fig['layout']['yaxis2'].update(title='')
# fig['layout']['yaxis3'].update(title='')
# fig['layout']['yaxis4'].update(title='')

fig.add_annotation(x=-0.07, y=0.5, xref='paper', yref='paper', showarrow=False, text='Distortion level',  font=dict(size=20),textangle=-90)
fig.add_annotation(x=1, y=0.5, xref='paper', yref='paper', showarrow=False, text='Log returns (%)',  font=dict(size=20),textangle=-90)

fig.show()
fig.write_image('./image/DL_eth.pdf')


In [20]:

fig = make_subplots(rows=3, cols=1, 
                    shared_xaxes=True, 
                    vertical_spacing=0)

fig['layout'].update(height=900, width=1200, 
                    title='',
                    showlegend=False,
                    font=dict(family='Times New Roman', size=16))

fig.append_trace(go.Scatter(
        y=eth_zoo1['return_y']*100, 
        x=eth_zoo1.index,
        line=dict(color='black', width=1)   
        ), row=1, col=1)

fig.append_trace(go.Scatter(
        y=eth_zoo1['fitted_conditional_stdev']*100, 
        x=eth_zoo1.index,
        line=dict(color='blue', width=1)   
        ), row=2, col=1)

fig.append_trace(go.Scatter(
        y=eth_zoo['fitted_conditional_stdev']*100,
        x=eth_zoo.index,
        line=dict(color='red', width=1)
        ), row=3, col=1)

fig['layout']['yaxis1'].update(title='Log return (%)',  title_standoff = 5,
                              showticklabels=True)
fig['layout']['yaxis2'].update(title='One-comp: St. dev.',  title_standoff = 20,
                              showticklabels=True)
fig['layout']['yaxis3'].update(title='Two-comp: St. dev.',  title_standoff = 20,
                              showticklabels=True)

fig['layout']['xaxis3'].update(title='',
                                dtick='M1',
                                tickformat='%b\n%Y',
                                showticklabels=True,
                                range=['2021-01-01','2023-01-10'])

fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False)

fig.update_layout({'plot_bgcolor': 'rgba(0,0,0,0)',
                'paper_bgcolor': 'rgba(0,0,0,0)'},
                font_color='black')

fig.show()
fig.write_image('./image/return_std.pdf')